# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [28]:
# import numpy and pandas

import numpy as np
import pandas as pd
import scipy.stats as st

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [5]:
# Your code here:

pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [9]:
# Your code here:

legendary = pokemon[pokemon.Legendary == True]
legendary

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
156,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,1,True
157,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
158,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,1,True
162,150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
163,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [10]:
regular = pokemon[pokemon.Legendary == False]
regular

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,711,GourgeistSuper Size,Ghost,Grass,494,85,100,122,58,75,54,6,False
788,712,Bergmite,Ice,NaN,304,55,69,85,32,35,28,6,False
789,713,Avalugg,Ice,NaN,514,95,117,184,44,46,28,6,False
790,714,Noibat,Flying,Dragon,245,40,30,35,45,40,55,6,False


In [11]:
number_of_legendary = len(legendary.index)
number_of_regular = len(regular.index)
print('count of legendary pokemons: ', number_of_legendary)
print('count of regular pokemons: ', number_of_regular)

count of legendary pokemons:  65
count of regular pokemons:  735


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [13]:
# Your code here:

mean_legendary = legendary['Total'].mean()
mean_regular = regular['Total'].mean()
std_legendary = legendary['Total'].std()
std_regular = regular['Total'].std()
print('mean of the total points for legendary pokemons: ', mean_legendary,
     'and standard deviation is: ', std_legendary)
print('mean of the total points for regular pokemons: ', mean_regular,
     'and standard deviation is: ', std_regular)

mean of the total points for legendary pokemons:  637.3846153846154 and standard deviation is:  60.93738905315346
mean of the total points for regular pokemons:  417.21360544217686 and standard deviation is:  106.76041745713022


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [52]:
# Your code here:

(t_stat, p_value) = st.ttest_ind(legendary['Total'], regular['Total'], equal_var = False)
# report
print('t_stat: %f, p_value: %f' % (t_stat, p_value))
if p_value > 0.05:
 print("Fail to reject the null hypothesis")
else:
 print("Reject the null hypothesis")

t_stat: 25.833574, p_value: 0.000000
Reject the null hypothesis


What do you conclude from this test? Write your conclusions below.

In [ ]:
# Your conclusions here:
'''

p-value is very low, what make us reject the null-hypothesis. ttest_ind is for the 
null hypothesis that 2 independent samples have identical average values.
so we can see that null hypothesis of this test doesn't fit us, so average values suppose to be
different (what we can see from calculating the mean before)

'''


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [32]:
# Your code here:

pokemon.groupby(['Type 1'])['Name'].count().sort_values(ascending = False)

Type 1
Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Ghost        32
Ground       32
Dragon       32
Dark         31
Poison       28
Fighting     27
Steel        27
Ice          24
Fairy        17
Flying        4
Name: Name, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [37]:
# Your code here:

water = pokemon[pokemon['Type 1'] == 'Water']
others = pokemon[pokemon['Type 1'] != 'Water']

mean_water = water['Total'].mean()
std_water = water['Total'].std()
mean_others = others['Total'].mean()
std_others = others['Total'].std()

print('the mean of total power water type: ', mean_water)
print('the mean of total power other types: ', mean_others)
print('the standard deviation of total power water type: ', std_water)
print('the standard deviation of total power other types: ', std_others)

the mean of total power water type:  430.45535714285717
the mean of total power other types:  435.85901162790697
the standard deviation of total power water type:  113.18826606431458
the standard deviation of total power other types:  121.09168230208066


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [51]:
# Your code here:

# h0: mean others > mean water
# h1: mean others < mean water

# i will use again ttest_ind because it tests if means of two samples are different or same
(t_stat, p_value) = st.ttest_ind(others['Total'], water['Total'])
# report
print('t_stat: %f, p_value: %f' % (t_stat, p_value))
if p_value > 0.05:
 print("Fail to reject the null hypothesis")
else:
 print("Reject the null hypothesis")

t_stat: 0.441855, p_value: 0.658714
Fail to reject the null hypothesis


Write your conclusion below.

In [ ]:
# Your conclusions here:

# our hypothesis test can't reject that the mean of total power of other pokemons greater 
# than the mean of total power of water pokemons

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [56]:
# Your code here:

# h0: two samples have identical average values
# h1: two samples have not identical average values 

attack_score = pokemon['Attack']
defense_score = pokemon['Defense']
(t_stat, p_value) = st.ttest_rel(attack_score, defense_score, axis = None)
# report
print('t_stat: %f, p_value: %f' % (t_stat, p_value))
if p_value > 0.05:
 print("Fail to reject the null hypothesis")
else:
 print("Reject the null hypothesis")

t_stat: 4.325566, p_value: 0.000017
Reject the null hypothesis


Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

# this test allows us to reject h0 - we reject that defense and attack scores have identical
# average values. So it says us - there is a statistically significant difference between them

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [58]:
# Your code here:

# h0: two samples have identical average values
# h1: two samples have not identical average values 

special_attack = pokemon['Sp. Atk']
special_defense = pokemon['Sp. Def']
(t_stat, p_value) = st.ttest_rel(special_attack, special_defense, axis = None)
# report
print('t_stat: %f, p_value: %f' % (t_stat, p_value))
if p_value > 0.05:
 print("Fail to reject the null hypothesis")
else:
 print("Reject the null hypothesis")

t_stat: 0.853986, p_value: 0.393369
Fail to reject the null hypothesis


Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

# in this case we failed the to reject h0 of ttest_rel. 
# we can aasume that those samples match, at least we can't reject this

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [63]:
# Your code here:
    
difference = pokemon['Sp. Def'] - pokemon['Sp. Atk']

# h0: the mean of difference is 0
# h1: the mean of difference is not 0

(t_stat, p_value) = st.ttest_1samp(difference, 0)
# report
print('t_stat: %f, p_value: %f' % (t_stat, p_value))
if p_value > 0.05:
 print("Fail to reject the null hypothesis")
else:
 print("Reject the null hypothesis")

t_stat: -0.853986, p_value: 0.393369
Fail to reject the null hypothesis


In [ ]:
# using ttest_1samp method with difference of samples we received the same result

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

